TSG119 - AD Post-deployment Checks
==================================

This notebook is designed to validate your BDC configuration after an AD
deployment. It will check the following:

1.  The existence of DNS entries for all endpoints with a `dnsName`
    attribute

    -   These DNS entries should be host records, not aliases (i.e. A
        records not CNAME records)

2.  The existence of well-known AD accounts and whether or not they are
    enabled.

3.  The existence of the expected SPNs

Prerequisites:
--------------

### General Prerequisites:

-   Install DNS Python
    -   In Azure Data Studio, click “Manage Packages” on the top bar
    -   Click “Add new”
    -   In the search field, type “dnspython” and click “search”
    -   Click Install

### Windows Prerequisites:

-   Install the Active Directory (AD) module for Powershell by following
    the instructions
    [here](https://docs.microsoft.com/en-us/powershell/module/addsadministration/?view=win10-ps).

### Linux Prerequisites:

-   Kinit as the BDC admin account

        kinit <user>

-   Install `adutil`

        wget -qO- https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
        sudo add-apt-repository "$(wget -qO- https://packages.microsoft.com/config/ubuntu/20.04/mssql-server-preview.list)"
        sudo apt-get update
        sudo apt-get install -y adutil

### Parameters

In [ ]:
configDirPath = ""

Read Configs
------------

First, the configuration files used to deploy the cluster must be
parsed.

In [ ]:
# If no values passed as parameters, prompt the user for values
print("Config directory path: ")

if not configDirPath or configDirPath == "":
  configDirPath = input().strip('"').rstrip('/').rstrip('\\')
else:
  print("")

print(configDirPath + "\n")

In [ ]:
# Define utility functions for dependency checks.
import importlib
import os
import platform
import subprocess
import shutil


# Function: cleanOutput
# 
# Description: Cleans output of a PowerShell command
# 
# Parameters: 
#   output - output to clean
# 
# Returns:
#   The decoded (UTF-8) output without extra white space.
#  
def cleanOutput(output):
  return output.strip().decode("UTF-8")


# Function: runSubprocessCommand
# 
# Parameters:
#   - executable: the executable to pass the command string
#     to.
#   - cmdStr: the string corresponding to the command
#     to run.
# 
# Returns: the output and errors of the execution
# of the command corresponding to the given
# command string.
def runSubprocessCommand(executable, cmd):
  process = subprocess.Popen(cmd, shell=True, executable=executable, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdOut, stdErr = process.communicate()
  return cleanOutput(stdOut), cleanOutput(stdErr)


# Function: runPowerShellCommand
# 
# Parameters:
#   - cmdStr: the string corresponding to the command
#     to run.
# 
# Returns: the output and errors of the execution
# of the PowerShell command corresponding to the
# given command string.
def runPowerShellCommand(cmdStr):
  return runSubprocessCommand(shutil.which('powershell'), cmdStr)


# Function: runBashCommand
#
# Parameters:
#   - cmdStr: the string corresponding to the command
#     to run.
#
# Returns: the output and errors of the execution
# of the bash command corresponding to the given
# command string.
def runBashCommand(cmdStr):
  return runSubprocessCommand('/bin/bash', cmdStr)


# Function: getDependencyError
# 
# Parameters:
#   - dependencyName: the name of the dependency that is missing
#     for running the notebook.
# 
# Returns: The string corresponding to the missing dependency error message. 
def getDependencyError(dependencyName):
  return "Error: missing dependency: {0}. Install it before running the notebook.".format(dependencyName)

# Function: adUserHasLoggedIn
#
# Parameters:
#
# Returns: True if an AD user has logged in. Else, false.
def adUserHasLoggedIn():

  klistOut, klistErr = runBashCommand('klist')

  if not klistOut or klistOut == '' or (klistErr and klistErr != ''):
    print('Error: You must kinit as an AD user before running this notebook.')
    return False

  return True

# Function: isADUtilInstalled
# 
# Parameters:
# 
# Returns: True if ADUtil is installed. Else, false.
def isADUtilInstalled():
  stdOut, stdErr = runBashCommand("whereis adutil")
  words = stdOut.split(':')

  res = len(words) == 2 and words[1] != ''

  if not res:
    print(getDependencyError('adutil'))

  return res


# Function: isPowerShellModuleInstalled
# 
# Parameters:
#   moduleName - the name of the module to look for
# 
# Returns: True if the PowerShell module is installed. Else, false.
def isPowerShellModuleInstalled(moduleName):
  res = False

  powerShellCmdStr = "(Get-Module -ListAvailable -Name '" + moduleName + "').Name"
  stdOut, stdErr = runPowerShellCommand(powerShellCmdStr)

  if stdOut == moduleName:
    res = True

  return res


# Function: isAdPowerShellModuleInstalled
# 
# Parameters:
# 
# Returns: True if the AD PowerShell module is installed. Else, false.
def isAdPowerShellModuleInstalled():
  res = isPowerShellModuleInstalled('ActiveDirectory')
  
  if not res:
    print(getDependencyError('ActiveDirectory PowerShell module'))

  return res


# Function: isPythonModuleInstalled
# 
# Parameters:
#   moduleName - the name of the module to look for
# 
# Returns: True if the Python module is installed. Else, false.
def isPythonModuleInstalled(moduleName):
  moduleSpec = importlib.util.find_spec(moduleName)
  return moduleSpec is not None


WINDOWS_OS_NAME = 'windows'
LINUX_OS_NAME = 'linux'


# Function: getHostOS
# 
# Parameters:
# 
# Returns: the name of the host operating system
# in lower case.
def getHostOS():
  return platform.system().lower()


HOST_OS_NAME = getHostOS()

:::

In [ ]:
# Define utility functions and declare global variables.
import json
from enum import Enum

# Parse the following elements of control.json and bdc.json
# 
# - From control.json
#     1. spec.endpoints[0].dnsName
#     2. spec.endpoints[1].dnsName
#     3. DOMAIN_DNS_NAME
#     4. OU_DISTINGUISHED_NAME
#     5. SUBDOMAIN
#     6. ACCOUNT_PREFIX
# 
# - From bdc.json
#     1. spec.resources.master.spec.endpoints[0].dnsName
#     2. spec.resources.gateway.spec.endpoints[0].dnsName
#     3. spec.resources.appproxy.spec.endpoints[0].dnsName
# 
# - From both control.json and bdc.json:
#     1. DEPLOYMENT_COMPONENTS_LIST (e.g. resources, services,
#     controller, service proxy).

NOTEBOOK_EXECUTION_SETUP_SUCCEEDED = False

# Devnote (saorozco): as of 5/29/2020 transient failures have
# been seen in the execution of the AD object checks and SPN
# checks of this notebook. As many other GCI tests running on
# the same container perform 'kinit' and 'kdestroy' operations,
# several of which are parallel tests, a possibility is that a race
# condition between different tests is ocurring, which is causing
# the Kerberos cache credential to be destroyed between 1. the
# 'kinit' operation performed for this notebook as part of the test
# setup, and 2. the last LDAP call that is made by this notebook
# through adutil. Another possibility is that DNS failures have
# been happening. For now, in case a transient failure occurs,
# the overall notebook execution will not be marked as a failure.
LOCAL_ERROR_IN_LDAP_CALL = False

controlJsonFileName = 'control.json'
loadedControlJson = False

bdcJsonFileName = 'bdc.json'
loadedBdcJson = False

# Enum for endpoint types
class EndpointType(Enum):
  Controller = 1
  ServiceProxy = 2
  Master = 3
  Gateway = 4
  AppProxy = 5

# Array of pairs <endpoint-dns-name (string), endpoint-type (EndpointType)> to check
dnsNamesAndTypesPairsArr = []

# List of components of the deployment (e.g. resources, services, controller, service proxy)
DEPLOYMENT_COMPONENTS_LIST = []

# The domain DNS name of the cluster deployment
DOMAIN_DNS_NAME = '<DOMAIN_DNS_NAME>'

# The organizational unit distinguished name of
# the cluster deployment
OU_DISTINGUISHED_NAME = '<OU_DISTINGUISHED_NAME>'

# The DNS subdomain to use internally
SUBDOMAIN = '<SUBDOMAIN>'

# Account prefix
ACCOUNT_PREFIX = '<ACCOUNT_PREFIX>'


In [ ]:
# Cell that finishes the notebook execution setup

# Parse config files

# Parse the following elements of control.json:
#     1. spec.endpoints[*].dnsName
#     2. DOMAIN_DNS_NAME
#     3. OU_DISTINGUISHED_NAME
#     4. SUBDOMAIN
#     5. ACCOUNT_PREFIX
#     6. DEPLOYMENT_COMPONENTS_LIST (controller, service proxy).
import json

# 1. Parse control.json
try:
  with open(os.path.join(configDirPath, controlJsonFileName)) as controlJsonFile:
    controlConfig = json.load(controlJsonFile)

    # Parse the domain DNS name of the cluster deployment
    DOMAIN_DNS_NAME = controlConfig['security']['activeDirectory']['domainDnsName']

    # Parse subdomain
    if 'subdomain' in controlConfig['security']['activeDirectory']:
      SUBDOMAIN = controlConfig['security']['activeDirectory']['subdomain']
    else:
      SUBDOMAIN = 'test'

    # Parse account prefix
    if 'accountPrefix' in controlConfig['security']['activeDirectory']:
      ACCOUNT_PREFIX = controlConfig['security']['activeDirectory']['accountPrefix']
    else:
      ACCOUNT_PREFIX = SUBDOMAIN

    # Parse the organizational unit distinguished name of the cluster deployment
    OU_DISTINGUISHED_NAME = controlConfig['security']['activeDirectory']['ouDistinguishedName']

    # Parse DNS endpoints
    if controlConfig['spec'] and controlConfig['spec']['endpoints']:
      parsedControllerEndpoint = False
      parsedServiceProxyEndpoint = False

      # Parse endpoints
      for endpoint in controlConfig['spec']['endpoints']:
        DEPLOYMENT_COMPONENTS_LIST.append({'name': endpoint['name'], 'replicas': 1, 'port': endpoint['port']})

        if endpoint['name'].lower() == 'controller':
          dnsNamesAndTypesPairsArr.append((endpoint['dnsName'], EndpointType.Controller))
          parsedControllerEndpoint = True
        elif endpoint['name'].lower() == 'serviceproxy':
          dnsNamesAndTypesPairsArr.append((endpoint['dnsName'], EndpointType.ServiceProxy))
          parsedServiceProxyEndpoint = True

      loadedControlJson = (parsedControllerEndpoint and parsedServiceProxyEndpoint)

except IOError:
  print("Error: could not read {0}.".format(os.path.join(configDirPath, controlJsonFileName)))
except KeyError:
  print("Error: missing settings in {0}. A key in the JSON file was not found.".format(os.path.join(configDirPath, controlJsonFileName)))
except IndexError:
  print("Error: missing settings in {0}. An index in the JSON file was not found.".format(os.path.join(configDirPath, controlJsonFileName)))

# Parse the following elements in bdc.json:
#     1. spec.resources.master.spec.endpoints[0].dnsName
#     2. spec.resources.gateway.spec.endpoints[0].dnsName
#     3. spec.resources.appproxy.spec.endpoints[0].dnsName
#     4. DEPLOYMENT_COMPONENTS_LIST (resources).
try:
  with open(os.path.join(configDirPath, bdcJsonFileName)) as bdcJsonFile:
    bdcConfig = json.load(bdcJsonFile)

    # Parse DNS names
    resourcesData = bdcConfig['spec']['resources']
    parsedMasterEndpoint = False
    parsedGatewayProxyEndpoint = False
    parsedAppProxyProxyEndpoint = False

    for masterInstanceData in resourcesData['master']['spec']['endpoints']:
      dnsNamesAndTypesPairsArr.append((masterInstanceData['dnsName'], EndpointType.Master))
      parsedMasterEndpoint = True

    for gatewayInstanceData in resourcesData['gateway']['spec']['endpoints']:
      dnsNamesAndTypesPairsArr.append((gatewayInstanceData['dnsName'], EndpointType.Gateway))
      parsedGatewayProxyEndpoint = True

    for appProxyInstanceData in resourcesData['appproxy']['spec']['endpoints']:
      dnsNamesAndTypesPairsArr.append((appProxyInstanceData['dnsName'], EndpointType.AppProxy))
      parsedAppProxyProxyEndpoint = True

    # Parse resources
    parsedResources = False

    for resourceName in bdcConfig['spec']['resources']:
      replicas = bdcConfig['spec']['resources'][resourceName]['spec']['replicas']
      DEPLOYMENT_COMPONENTS_LIST.append({'name': resourceName, 'replicas': replicas})
      parsedResources = True

    loadedBdcJson = (parsedMasterEndpoint and parsedGatewayProxyEndpoint and parsedAppProxyProxyEndpoint) and (parsedResources)

except IOError:
  print("Error: could not read {0}.".format(os.path.join(configDirPath, bdcJsonFileName)))
except KeyError:
  print("Error: missing settings in {0}. A key in the JSON file was not found.".format(os.path.join(configDirPath, bdcJsonFileName)))
except IndexError:
  print("Error: missing settings in {0}. An index in the JSON file was not found.".format(os.path.join(configDirPath, bdcJsonFileName)))


# Function: endpointTypeDataExists
# 
# Description: Checks whether a given endpoint type
#              exists in the array of parsed endpoints
# 
# Parameters: 
#   dnsNamesAndTypesPairsArr - the array of parsed endpoints
#   typeToLookFor - the type of endpoint to look for
# 
# Returns:
#   True if the endpoint type exists. Else, false.
def endpointTypeDataExists(dnsNamesAndTypesPairsArr, typeToLookFor):
  res = False

  for endpointName, endpointType in dnsNamesAndTypesPairsArr:
    if endpointType == typeToLookFor:
      res = True
      break

  return res

# Print appropriate error messages to help troubleshooting. 
if not loadedControlJson and not loadedBdcJson:
  print("Failed to load config files. Please check that the provided directory has both {0} and {1}.".format(controlJsonFileName, bdcJsonFileName))
elif not loadedControlJson or not loadedBdcJson:

  if not endpointTypeDataExists(dnsNamesAndTypesPairsArr, EndpointType.Controller):
    print("Failed to find endpoint defined for controller in control.json")

  elif not endpointTypeDataExists(dnsNamesAndTypesPairsArr, EndpointType.ServiceProxy):
    print("Failed to find endpoint defined for service proxy in control.json")

  elif not endpointTypeDataExists(dnsNamesAndTypesPairsArr, EndpointType.Master):
    print("Failed to find endpoint defined for master in bdc.json")

  elif not endpointTypeDataExists(dnsNamesAndTypesPairsArr, EndpointType.Gateway):
    print("Failed to find endpoint defined for gateway in bdc.json")

  elif not endpointTypeDataExists(dnsNamesAndTypesPairsArr, EndpointType.AppProxy):
    print("Failed to find endpoint defined for app proxy in bdc.json")

  elif not parsedResources:
    print("Failed to parse resources section in {0}".format(bdcJsonFileName))

  elif not parsedServices:
    print("Failed to parse services section in {0}".format(bdcJsonFileName))

  else:
    print("Failed to load config files. Please check that the provided directory has both {0} and {1}.".format(controlJsonFileName, bdcJsonFileName))
else:

  # On Linux systems, check that an AD user has logged in
  if getHostOS() == LINUX_OS_NAME:
      NOTEBOOK_EXECUTION_SETUP_SUCCEEDED = adUserHasLoggedIn()

      if not NOTEBOOK_EXECUTION_SETUP_SUCCEEDED:
        LOCAL_ERROR_IN_LDAP_CALL = True

if NOTEBOOK_EXECUTION_SETUP_SUCCEEDED:
  print("Success\nThe following files were successfully parsed:\n- {0}\n- {1}".format(controlJsonFileName, bdcJsonFileName))

::: \#\# DNS Checks

This checks that the correct DNS objects have been set for the cluster.
For example, host records have been created for all externally facing
FQDNs in the cluster.

In [ ]:
# Define DNS utility functions.
import sys
import socket
import dns.resolver


# Function: dnsEntryExists
# 
# Description: Checks whether a DNS entry exists.
# 
# Parameters: 
#   endpointDnsName - the name to check
# 
# Returns:
#   True if the DNS entry exists. Else, false.
def dnsEntryExists(endpointDnsName):
  entryExists = False

  try:
    ipAddress = socket.gethostbyname(endpointDnsName)
    entryExists = True
  except:
    pass

  return entryExists


# Function: dnsEntryIsHostRecord
# 
# Description: Checks whether a DNS entry is a host record.
# 
# Parameters: 
#   endpointDnsName - the name to check
# 
# Returns:
#   True if the DNS entry is a host record. Else, false.
def dnsEntryIsHostRecord(endpointDnsName):
  entryIsHostRecord = False

  try:
    result = dns.resolver.resolve(
      qname = endpointDnsName,
      rdtype = dns.rdatatype.A # DNS "A" record
      )

    for ipval in result:
      assert (len(ipval.to_text()) > 0)
      entryIsHostRecord = True

  except:
    pass

  return entryIsHostRecord

:::

In [ ]:
# Perform DNS checks.

dnsChecksSucceeded = False

# Function: dnsSetupIsCorrect
#
# Description:
# Drives the execution of the DNS test. Displays
# the list of both the existing and missing entries,
# if any.
#
# Parameters:
#
# Returns:
#   True if the required DNS setup is correct. Else, false.
#   Displays the list of both the existing and missing entries,
#   if any.
def dnsSetupIsCorrect():

  dnsPassedChecksCount = 0

  # Iterate through the list of DNS names
  for endpointName, endpointType in dnsNamesAndTypesPairsArr:
    if dnsEntryExists(endpointName):
      if dnsEntryIsHostRecord(endpointName):
        dnsPassedChecksCount += 1
        print("Success: DNS entry for {0} exists and is a host record.".format(endpointName))
      else:
        print("Error: DNS entry for {0} exists, but is not a host record.".format(endpointName))
    else:
      print("Error: DNS entry for {0} does not exist.".format(endpointName))

  return dnsPassedChecksCount == len(dnsNamesAndTypesPairsArr)

shouldRunTests = False

try:
  NOTEBOOK_EXECUTION_SETUP_SUCCEEDED
  shouldRunTests = True
except NameError:
  print("Error: The first cells of this notebook, which perform the notebook execution setup, must be run before this cell.")

if shouldRunTests:
  dnsChecksSucceeded = dnsSetupIsCorrect()

# Summary of the test
if dnsChecksSucceeded:
  print("\nDNS checks were successful.")
else:
  print("\nDNS checks failed.")


::: \#\# AD Account Checks

This verifies that all AD accounts the cluster should have created
exist.

In [ ]:
# Define utility functions. 
import subprocess
import shutil
import platform


# Enum for AD account types
class AccountType(Enum):
  User = 1
  Group = 2
  OU = 3


# Function: getAdObjectLookupPowerShellCommandStr
# 
# Parameters:
#   - accountName: the account to check
#   - accountType: the type of account to check
# 
# Returns: the string of the appropriate powershell
#          command for looking up the AD object
def getAdObjectLookupPowerShellCommandStr(accountName, accountType):
  powerShellCommandStr = ""

  if accountType == AccountType.Group:
    powerShellCommandStr = "Get-ADGroup -Filter 'samAccountName -Like \\\"{0}\\\"'".format(accountName)
  elif accountType == AccountType.User:
    powerShellCommandStr = "Get-ADUser -Filter 'samAccountName -Like \\\"{0}\\\"'".format(accountName)
  elif accountType == AccountType.OU:
    powerShellCommandStr = "Get-ADOrganizationalUnit -Filter 'distinguishedname -Like \\\"{0}\\\"'".format(accountName)

  return powerShellCommandStr


# Function: parsePowerShellCommandOutput
# 
# Parameters:
#   - stdOut: standard output from PowerShell command execution
#   - stdErr: standard error from PowerShell command execution
# 
# Returns: 
#   - exists: a Boolean value indicating whether the AD object exists
#   - error: the error obtained during the PowerShell command execution,
#            if any.
def parsePowerShellCommandOutput(stdOut, stdErr):
  exists = False
  error = ''

  if stdErr and len(str(stdErr)) > 0:
    # If there is an error, return it.
    # 
    exists = False
    error = stdErr
  elif len(str(stdOut)) > 0:
    # If anything is printed to stdout, then an object has been found.
    #
    exists = True
    error = ''
  else:
    # No object found and no errors means the object doesn't exist.
    #
    exists = False
    error = ''

  return exists, error


# Function: dependenciesAreInstalledWindows
# 
# Parameters:
# 
# Returns: True if the Windows dependencies
# for the checks to be done are installed.
# Else, false.
def dependenciesAreInstalledWindows():
  return isAdPowerShellModuleInstalled()


# Function: dependenciesAreInstalledLinux
# 
# Parameters:
# 
# Returns: True if the Linux dependencies
# for the checks to be done are installed.
# Else, false.
def dependenciesAreInstalledLinux():
  return isADUtilInstalled()


# Function: dependenciesAreInstalled
# 
# Parameters:
# 
# Returns: True if the Linux dependencies
# for the checks to be done are installed.
# Else, false.
def dependenciesAreInstalled():
  res = False

  if HOST_OS_NAME == WINDOWS_OS_NAME:
    res = dependenciesAreInstalledWindows()
  elif HOST_OS_NAME == LINUX_OS_NAME:
    res = dependenciesAreInstalledLinux()

  return res

:::

In [ ]:
# Define AD-related functions 

# Define the lists of well-known groups and users.
WELL_KNOWN_GROUPS = [
  "dmsvc",
  "desvc"
]

WELL_KNOWN_USERS = [
  "apst",
  "ctrl",
  "dmc*-*",
  "dmmp-*",
  "dec*-*",
  "demp-*",
  "hdt*-*",
  "hdnn-*",
  "hvsh-*",
  "htgw",
  "htnn",
  "htsh",
  "htt*",
  "htzk",
  "jnzk-*",
  "kmnn-*",
  "knox-*",
  "ldap",
  "lvsh-*",
  "sqc*",
  "sqc*-*",
  "sqd*",
  "sqmp",
  "sqmp-*",
  "sqs*",
  "ngxm",
  "shsh-*",
  "ynt*-*",
  "yrsh-*"
]


# Function: adUserAccountIsEnabledWindows
# 
# Parameters:
#   - accountName: the account to check
# 
# Returns:
#   - True if the AD user account is enabled. Else, false.
#   - The error message obtained while performing the account check
def adUserAccountIsEnabledWindows(accountName):
  isEnabled = False
  error = 'Function adUserAccountIsEnabledWindows() did not finish'
  powerShellCommandStr = "(Get-ADUser -Filter 'samAccountName -Like \\\"{0}\\\"').enabled".format(accountName)
  stdOut, stdErr = runPowerShellCommand(powerShellCommandStr)

  # Parse output.
  if stdErr and stdErr != '':
    error = stdErr
  elif not stdOut or stdOut == '':
    # If nothing is returned the user probably does not have permissions.
    # Print a warning, but do not fail the test.
    isEnabled = True
    error = "Warning: Missing permissions to check if user '{0}' is enabled. Please manually check this or try running Azure Data Studio as administrator".format(accountName)
  else:
    # Check that the account disabled flag isn't set.
    # Note: if 'accountName' is a regex that matches
    # more than one account, 'stdOut' will contain more
    # than one line. In that case, for now consider
    # only the first line of the output.
    ans = stdOut.split("\n")[0].lower().strip()
    isEnabled = ans == 'true'
    error = ''
  
  return isEnabled, error


# Function: adAccountExistsWindows
# 
# Parameters:
#   - accountName: the account to check
#   - accountType: the type of account to check
# 
# Returns: 
#   - If it's a user account, true if the account exists and is
#     enabled. If it's not a user account, true only if it exists.
#     Else, false.
#   - The error message obtained while performing the account check.
def adAccountExistsWindows(accountName, accountType):
  adAccountExists = False
  adAccountIsEnabled = False
  error = "The execution of adAccountExistsWindows({0}, {1}) didn't finish".format(accountName, accountType)

  try:
    # First, check whether the account exists.
    powerShellCommandStr = getAdObjectLookupPowerShellCommandStr(accountName, accountType)
    stdOut, stdErr = runPowerShellCommand(powerShellCommandStr)
    adAccountExists, error = parsePowerShellCommandOutput(stdOut, stdErr)

    # If the account exists, then check that it is enabled as well.
    if accountType == AccountType.User and adAccountExists:
      adAccountIsEnabledStdOut, adAccountIsEnabledStdErr = adUserAccountIsEnabledWindows(accountName)
      
      if adAccountIsEnabledStdOut:
        adAccountIsEnabled = True
      else:
        error += "\nError: AD user account {0} is not enabled.".format(accountName)
        error += adAccountIsEnabledStdErr

  except Exception as e:
    adAccountExists = False
    print("Exception: " + str(e))

  return (accountType == AccountType.User and adAccountExists and adAccountIsEnabled) or (adAccountExists and accountType != AccountType.User), error


# Function: adAccountExistsLinux
# 
# Parameters:
#   - accountName: the account to check
# 
# Returns: 
#   - True if the AD account exists. Else, false.
#   - The error message obtained while performing the account check
def adAccountExistsLinux(accountName):
  exists = False
  error = ''
  stdOut, stdErr = runBashCommand("adutil account exists -n {0}".format(accountName))

  resLines = stdOut.split('\n')
  strRes = resLines[0].lower()

  # Parse output
  if strRes == 'true':
    exists = True
  else:
    error = stdOut + "\n" + stdErr

  return exists, error


# Function: adAccountsByTypeExist
# 
# Parameters:
#   - accountList: the list of accounts to check
#   - accountType: the type of accounts to check
# 
# Returns: True if the AD accounts exist. Else, false.
def adAccountsByTypeExist(accountList, accountType):
  global LOCAL_ERROR_IN_LDAP_CALL
  allAdAccountsExist = True

  for staticAccountName in accountList:
    account = ACCOUNT_PREFIX + '-' + staticAccountName
    exists = False
    error = ''

    if HOST_OS_NAME == WINDOWS_OS_NAME:
      exists, error = adAccountExistsWindows(account, accountType)
    elif HOST_OS_NAME == LINUX_OS_NAME:
      exists, error = adAccountExistsLinux(account)

    if not exists or error != '':
      allAdAccountsExist = False

      if error != '':
        print("Error looking up AD account '{0}'.\nDetails:\n{1}".format(account, error))

        if "Failed to perform LDAP search" in error and "Local error" in error:
          LOCAL_ERROR_IN_LDAP_CALL = True
      else:
        print("Account {0} does not exist, but it should.".format(account))

      break

  return allAdAccountsExist


# Function: adAccountsExist
# 
# Parameters:
# 
# Returns: True if the well-known AD accounts exist and the users
# are enabled. Else, false.
def adAccountsExist():
  return adAccountsByTypeExist(WELL_KNOWN_GROUPS, AccountType.Group) \
      and adAccountsByTypeExist(WELL_KNOWN_USERS, AccountType.User)

:::

In [ ]:
# Perform AD accounts checks.
import subprocess
import shutil
import platform

shouldRunTests = False
adAccountsChecksSucceeded = False

try:
  NOTEBOOK_EXECUTION_SETUP_SUCCEEDED
  shouldRunTests = True
except NameError:
  print("Error: The first cells of this notebook, which perform the notebook execution setup, must be run before this cell.")

if shouldRunTests:
  adAccountsChecksSucceeded = dependenciesAreInstalled() and adAccountsExist()

# Summary of the test
if adAccountsChecksSucceeded:
  print("AD accounts checks were successful.")
else:
  print("Error. AD accounts checks failed.")

SPN Checks
----------

Check that all service principal names (SPNs) the cluster should have
created exist.

In [ ]:
# Define constants and the ServicePrincipalNameBuilder class

DEPLOYMENT_COMPONENT_NAME_DICT_KEY = 'name'
DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY = 'replicas'
DEPLOYMENT_COMPONENT_PORT_DICT_KEY = 'port'


# Class: ServicePrincipalNameBuilder
# 
# Class that creates SPNs considering building blocks
# and combinations of them. The building blocks that
# are currently used are the following:
# 
# 1. Krb service class 
# 2. user account
# 3. pool ID
# 4. pod ID
# 5. subdomain and domain
# 6. port
class ServicePrincipalNameBuilder(object):

  # Constructor
  # 
  # Parameters:
  #   - spnBuilderAttributesList: the list of the required
  #   attributes to create the SPNs (list of SpnMetaData objects)
  def __init__(self, spnBuilderAttributesList=None):
    if spnBuilderAttributesList.serviceClassList == None or spnBuilderAttributesList.userAccount == None:
      print("Error: Missing service class or user account attribute when building SPNs")
    else:
      self.krbServiceClassArr = spnBuilderAttributesList.serviceClassList
      self.userAccount = spnBuilderAttributesList.userAccount
      self.poolId = spnBuilderAttributesList.poolId
      self.numPods = spnBuilderAttributesList.numPods
      self.domainArr = [ SUBDOMAIN + '.' + DOMAIN_DNS_NAME ]
      self.portArr = spnBuilderAttributesList.portList

    # Rule of thumb:
    # 
    # Whenever a port is included as a building block to
    # generate SPNs, we also generate the corresponding
    # SPNs that do not include the port. To accomplish the
    # latter, an empty string is added to the list storing
    # the port.
    # 
    self.addEmptyStringToList(self.portArr)


  # Function: addEmptyStringToList
  # 
  # Parameters:
  #   - list: the list to which an empty string is to be added
  # 
  # Returns: None
  def addEmptyStringToList(self, list):
    if list \
        and len(list) > 0 \
        and len(str(list[0])) > 0:
      list += [""]


  # Function: getAllSPNs
  # 
  # Parameters:
  # 
  # Returns: the list of strings corresponding to all possible
  # SPNs that can be created given the "building blocks" that
  # are stored as members of this class.
  def getAllSPNs(self):
    # Each SPN has the following structure:
    # krbServiceClass/userAccount[-][poolID][-podID].subdomain.domain[:port]
    spnsList = []

    # Add the Krb service class building block
    for serviceAccount in self.krbServiceClassArr:
      stack = []
      stack.append(serviceAccount)
      stack.append("/")

      # Add the user account building block
      stack.append(self.userAccount)

      # Add the pool ID building block
      if self.poolId and len(self.poolId) > 0:
        stack.append("-")
        stack.append(self.poolId)

      # Add the pod ID building block
      podIdRange = [-1]

      if self.numPods:
        podIdRange = range(self.numPods)

      for podId in podIdRange:
        if podId >= 0:
          stack.append("-")
          stack.append(str(podId))

        # Add the domain building block
        for domain in self.domainArr:
          if len(domain) > 0:
            stack.append(".")
            stack.append(domain)

          # Add the port building block
          portList = [""]

          if self.portArr:
            portList = self.portArr

          for port in portList:
            if len(port) > 0:
              stack.append(":")
              stack.append(port)

            # Finally, create the SPN by joining
            # into a string the current elements
            # in the stack and append the SPN
            # to the list that is to be returned
            # by this function.
            currSPN = ''.join(stack)
            spnsList.append(currSPN)

            if len(port) > 0:
              # Pop the port and the colon (':')
              stack.pop()
              stack.pop()
          
          if len(domain) > 0:
            # Pop the domain and the dot ('.')
            stack.pop()
            stack.pop()
        
        if podId >= 0:
          # Pop the pod id and the dash ('-')
          stack.pop()
          stack.pop()
          
      if self.poolId and len(self.poolId) > 0:
        # Pop the pool id and the dash ('-')
        stack.pop()
        stack.pop()

      # Pop the user account
      stack.pop()

    return spnsList

In [ ]:
# Define structures of expected SPNs


# Class which holds the metadata about a service's SPNs
class SpnMetaData(object):
  def __init__(self, serviceClassList, userAccount, portList, poolId=None, numPods=None):
    self.serviceClassList = serviceClassList
    self.userAccount = userAccount
    self.portList = portList
    self.poolId = poolId
    self.numPods = numPods


# Function: getControllerSpnBuildersAttributesList
# 
# Parameters:
#   - port: the port the controller HTTP service will run on 
# 
# Returns: the list of existing SpnMetaData objects
def getControllerSpnBuildersAttributesList(port):
  return [
    SpnMetaData(
      ["HTTP"],
      "control",
      [str(port)]),
    SpnMetaData(
      ["HTTP"],
      "controller-svc",
      ["443"])
  ]

  return attributesDicts


# Function: getServiceProxySpnBuildersAttributesList
# 
# Parameters:
#   - port: the port the service proxy HTTP service will run on 
# 
# Returns: the list of existing SpnMetaData objects
def getServiceProxySpnBuildersAttributesList(port):
  return [
    SpnMetaData(
      ["HTTP"],
      "monitor",
      [str(port)]),
    SpnMetaData(
      ["HTTP"],
      "mgmtproxy-svc",
      ["8443"])
  ]


# Function: getZooKeeperSpnBuildersAttributesList
# 
# Parameters:
#   - numReplicas: number of zookeeper replicas
#   - deploymentComponentNamePrefix: prefix of the pod name
#   - deploymentComponentNameSuffix: pod name suffix (e.g. pool ID)
# 
# Returns: the list of existing SpnMetaData objects
def getZooKeeperSpnBuildersAttributesList(numReplicas, deploymentComponentNamePrefix, deploymentComponentNameSuffix):
  return [
    SpnMetaData(
      ["HTTP", "jn"],
      deploymentComponentNamePrefix,
      None,
      deploymentComponentNameSuffix,
      numReplicas)
  ]

  return attributesDicts


# Function: getSparkheadSpnBuildersAttributesList
# 
# Parameters:
#   - numReplicas: number of sparkhead replicas
#   - deploymentComponentNamePrefix: prefix of the pod name
#   - deploymentComponentNameSuffix: pod name suffix (e.g. pool ID)
# 
# Returns: the list of existing SpnMetaData objects
def getSparkheadSpnBuildersAttributesList(numReplicas, deploymentComponentNamePrefix, deploymentComponentNameSuffix):
  return [
    SpnMetaData(
      ["hive", "HTTP", "livy", "sph", "yarn"],
      deploymentComponentNamePrefix,
      None,
      deploymentComponentNameSuffix,
      numReplicas)
  ]

# Function: getSparkSpnBuildersAttributesList
# 
# Parameters:
#   - numReplicas: number of spark replicas
#   - deploymentComponentNamePrefix: prefix of the pod name
#   - deploymentComponentNameSuffix: pod name suffix (e.g. pool ID)
# 
# Returns: the list of existing SpnMetaData objects
def getSparkSpnBuildersAttributesList(numReplicas, deploymentComponentNamePrefix, deploymentComponentNameSuffix):
  return [
    SpnMetaData(
      ["HTTP", "yarn"],
      deploymentComponentNamePrefix,
      None,
      deploymentComponentNameSuffix,
      numReplicas)
  ]

# Function: getMasterSpnBuildersAttributesList
# 
# Parameters:
#   - numReplicas: number of master replicas
#   - deploymentComponentNamePrefix: prefix of the pod name
#   - deploymentComponentNameSuffix: pod name suffix (e.g. pool ID)
# 
# Returns: the list of existing SpnMetaData objects
def getMasterSpnBuildersAttributesList(numReplicas, deploymentComponentNamePrefix, deploymentComponentNameSuffix):
  return [
    SpnMetaData(
      ["DWDMSSvc"],
      deploymentComponentNamePrefix,
      ["16450"],
      deploymentComponentNameSuffix,
      numReplicas),
    SpnMetaData(
      ["DWENGSvc"],
      deploymentComponentNamePrefix,
      ["17001"],
      deploymentComponentNameSuffix,
      numReplicas),
    SpnMetaData(
      ["MSSQLSvc"],
      deploymentComponentNamePrefix,
      ["1433", "1533"],
      deploymentComponentNameSuffix,
      numReplicas),
    SpnMetaData(
      ["MSSQLSvc"],
      "mssql",
      ["31433"]),
    SpnMetaData(
      ["MSSQLSvc"],
      deploymentComponentNamePrefix,
      ["1433", "1533"],
      "p-svc"),
    SpnMetaData(
      ["MSSQLSvc"],
      deploymentComponentNamePrefix,
      ["1433", "1533"],
      "svc")
  ]


# Function: getGatewaySpnBuildersAttributesList
# 
# Parameters:
#   - numReplicas: number of gateway replicas
#   - deploymentComponentNamePrefix: prefix of the pod name
#   - deploymentComponentNameSuffix: pod name suffix (e.g. pool ID)
# 
# Returns: the list of existing SpnMetaData objects
def getGatewaySpnBuildersAttributesList(numReplicas, deploymentComponentNamePrefix):
  return [
    SpnMetaData(
      ["HTTP"],
      deploymentComponentNamePrefix,
      None,
      None,
      numReplicas),
    SpnMetaData(
      ["HTTP"],
      "knox",
      None),
    SpnMetaData(
      ["knox"],
      deploymentComponentNamePrefix,
      ["8080"],
      None,
      numReplicas)
  ]


# Define functions to generate SPNs attribute dictionaries for storage, data, nmnode and compute.


# Function: getStorageSpnBuildersAttributesList
# 
# Parameters:
#   - numReplicas: number of storage pool replicas
#   - deploymentComponentNamePrefix: prefix of the pod name
#   - deploymentComponentNameSuffix: pod name suffix (e.g. pool ID)
# 
# Returns: the list of existing SpnMetaData objects
def getStorageSpnBuildersAttributesList(numReplicas, deploymentComponentNamePrefix, deploymentComponentNameSuffix):
  return [
    SpnMetaData(
      ["hdfs", "HTTP", "yarn"],
      deploymentComponentNamePrefix,
      None,
      deploymentComponentNameSuffix,
      numReplicas),
    SpnMetaData(
      ["MSSQLSvc"],
      deploymentComponentNamePrefix,
      ["1433"],
      deploymentComponentNameSuffix,
      numReplicas),
    SpnMetaData(
      ["MSSQLSvc"],
      deploymentComponentNamePrefix,
      ["1433"],
      str(deploymentComponentNameSuffix + "-svc"))
  ]


# Function: getDataSpnBuildersAttributesList
# 
# Parameters:
#   - numReplicas: number of data pool replicas
#   - deploymentComponentNamePrefix: prefix of the pod name
#   - deploymentComponentNameSuffix: pod name suffix (e.g. pool ID)
# 
# Returns: the list of existing SpnMetaData objects
def getDataSpnBuildersAttributesList(numReplicas, deploymentComponentNamePrefix, deploymentComponentNameSuffix):
  return [
    SpnMetaData(
      ["MSSQLSvc"],
      deploymentComponentNamePrefix,
      ["1433"],
      deploymentComponentNameSuffix,
      numReplicas),
    SpnMetaData(
      ["MSSQLSvc"],
      deploymentComponentNamePrefix,
      ["1433"],
      str(deploymentComponentNameSuffix + "-svc"))
  ]


# Function: getNmnodeSpnBuildersAttributesList
# 
# Parameters:
#   - numReplicas: number of name node replicas
#   - deploymentComponentNamePrefix: prefix of the pod name
#   - deploymentComponentNameSuffix: pod name suffix (e.g. pool ID)
# 
# Returns: the list of existing SpnMetaData objects
def getNmnodeSpnBuildersAttributesList(numReplicas, deploymentComponentNamePrefix, deploymentComponentNameSuffix):
  return [
    SpnMetaData(
      ["hdfs", "HTTP", "kms"],
      deploymentComponentNamePrefix,
      None,
      deploymentComponentNameSuffix,
      numReplicas)
  ]


# Function: getComputeSpnBuildersAttributesList
# 
# Parameters:
#   - numReplicas: number of compute pool replicas
#   - deploymentComponentNamePrefix: prefix of the pod name
#   - deploymentComponentNameSuffix: pod name suffix (e.g. pool ID)
# 
# Returns: the list of existing SpnMetaData objects
def getComputeSpnBuildersAttributesList(numReplicas, deploymentComponentNamePrefix, deploymentComponentNameSuffix):
  return [
    SpnMetaData(
      ["DWDMSSvc"],
      deploymentComponentNamePrefix,
      ["16450"],
      deploymentComponentNameSuffix,
      numReplicas),
    SpnMetaData(
      ["DWENGSvc"],
      deploymentComponentNamePrefix,
      ["17001"],
      deploymentComponentNameSuffix,
      numReplicas),
    SpnMetaData(
      ["MSSQLSvc"],
      deploymentComponentNamePrefix,
      ["1433"],
      deploymentComponentNameSuffix,
      numReplicas),
    SpnMetaData(
      ["MSSQLSvc"],
      deploymentComponentNamePrefix,
      ["1433"],
      deploymentComponentNameSuffix + "-svc")
  ]


:::

In [ ]:
# Define driver functions to generate the expected SPNs.

# Function: getSpnBuildersAttributesList
# 
# Description: gets the list of SPN metadata structures for the input
# component. Each element in the list is an instance of SpnMetaData
# 
# Parameters:
#   - deploymentComponent: the BDC deployment component (e.g. resource,
#     service, monitor or app proxy) whose associated list of dictionaries
#     of ServicePrincipalNameBuilder attributes we want to obtain.
# 
# Returns: the lists of SpnMetaDataObjects for generating SPNs for this
# deployment component
def getSpnBuildersAttributesList(deploymentComponent):
  # List of dictionaries to return
  attributesList = []

  # The name of the deployment component could be something like
  # 'zookeper' or 'storage-0'. In the latter case, we want to
  # strip off the pool id and get a prefix of the component name.
  deploymentComponentNamePrefix = deploymentComponent[DEPLOYMENT_COMPONENT_NAME_DICT_KEY].split('-')[0]

  # The suffix corresponds to the pool ID of the deployment component,
  # if the pool exists. Else, the suffix will be an empty string
  deploymentComponentNameSuffix = ''

  if len(deploymentComponent[DEPLOYMENT_COMPONENT_NAME_DICT_KEY].split('-')) > 1:
    deploymentComponentNameSuffix = deploymentComponent[DEPLOYMENT_COMPONENT_NAME_DICT_KEY].split('-')[1]

  # Populate the list of dictionaries of attributes which will
  # be used to create instances of ServicePrincipalNameBuilder
  # according to the deployment component type.

  if deploymentComponentNamePrefix == 'Controller':
    attributesList += getControllerSpnBuildersAttributesList(deploymentComponent[DEPLOYMENT_COMPONENT_PORT_DICT_KEY])
  elif deploymentComponentNamePrefix == 'ServiceProxy':
    attributesList += getServiceProxySpnBuildersAttributesList(deploymentComponent[DEPLOYMENT_COMPONENT_PORT_DICT_KEY])
  elif deploymentComponentNamePrefix == 'zookeeper':
    attributesList += getZooKeeperSpnBuildersAttributesList(
      deploymentComponent[DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY],
      deploymentComponentNamePrefix,
      deploymentComponentNameSuffix)
  elif deploymentComponentNamePrefix == 'sparkhead':
    attributesList += getSparkheadSpnBuildersAttributesList(
      deploymentComponent[DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY],
      deploymentComponentNamePrefix,
      deploymentComponentNameSuffix)
  elif deploymentComponentNamePrefix == 'spark':
    attributesList += getSparkSpnBuildersAttributesList(
      deploymentComponent[DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY],
      deploymentComponentNamePrefix,
      deploymentComponentNameSuffix)
  elif deploymentComponentNamePrefix == 'master':
    attributesList += getMasterSpnBuildersAttributesList(
      deploymentComponent[DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY],
      deploymentComponentNamePrefix,
      deploymentComponentNameSuffix)
  elif deploymentComponentNamePrefix == 'gateway':
    attributesList += getGatewaySpnBuildersAttributesList(
      deploymentComponent[DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY],
      deploymentComponentNamePrefix)
  elif deploymentComponentNamePrefix == 'storage':
    attributesList += getStorageSpnBuildersAttributesList(
      deploymentComponent[DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY],
      deploymentComponentNamePrefix,
      deploymentComponentNameSuffix)
  elif deploymentComponentNamePrefix == 'data':
    attributesList += getDataSpnBuildersAttributesList(
      deploymentComponent[DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY],
      deploymentComponentNamePrefix,
      deploymentComponentNameSuffix)
  elif deploymentComponentNamePrefix == 'nmnode':
    attributesList += getNmnodeSpnBuildersAttributesList(
      deploymentComponent[DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY],
      deploymentComponentNamePrefix,
      deploymentComponentNameSuffix)
  elif deploymentComponentNamePrefix == 'compute':
    attributesList += getComputeSpnBuildersAttributesList(
      deploymentComponent[DEPLOYMENT_COMPONENT_REPLICAS_DICT_KEY],
      deploymentComponentNamePrefix,
      deploymentComponentNameSuffix)

  return attributesList


# Function: getExpectedSPNs
# 
# Parameters:
#   - deploymentComponentsList: the list of deployment components
#     from which we will generate the expected SPNs.
# 
# Returns: the list of expected SPNs after the BDC deployment.
def getExpectedSPNs(deploymentComponentsList):
  expectedSPNsList = []

  for deploymentComponent in deploymentComponentsList:
    # Get the lists of required attributes for
    # the instances of ServicePrincipalNameBuilder
    # corresponding to that deployment component
    spnBuildersAttributesList = getSpnBuildersAttributesList(deploymentComponent)

    # Traverse the dictionaries of ServicePrincipalNameBuilder
    # attributes required to create the instances of 
    # ServicePrincipalNameBuilder and generate the expected SPNs. 
    for attr in spnBuildersAttributesList: 
      componentSPNs = ServicePrincipalNameBuilder(attr).getAllSPNs()
      expectedSPNsList += componentSPNs

  return expectedSPNsList

:::

In [ ]:
# Define functions to list the existing SPNs.


# Function: getExistingSPNsOfAdAccountLinux
# 
# Parameters:
#   - adAccountName: the name of the Active Directory 
#     account whose SPNs we want to obtain. 
# 
# Returns: the list of existing SPNs of the given
#           Active Directory account.
def getExistingSPNsOfAdAccountLinux(adAccountName):
  global LOCAL_ERROR_IN_LDAP_CALL
  existingSPNsOfAdAccount = []

  adutilShowSPNsCommandStr = "adutil spn show -n '{0}'".format(adAccountName)
  stdOut, stdErr = runBashCommand(adutilShowSPNsCommandStr)

  if "Failed to retrieve distinguished name of '{0}'".format(adAccountName) in stdOut:
    print("Error. The call to get SPNs of the previously-existing account '{0}' failed because "
          "the account was not found anymore or the Kerberos credential cache was destroyed before "
          "the SPNs could be retrieved.".format(adAccountName))

    LOCAL_ERROR_IN_LDAP_CALL = True

  else:
    existingSPNsOfAdAccount = stdOut.split("\n")

    # Delete extra whitespace in the strings.
    existingSPNsOfAdAccount = [spn.strip() for spn in existingSPNsOfAdAccount]
  
  return existingSPNsOfAdAccount


# Function: getExistingSPNsOfAdAccountWindows
# 
# Parameters:
#   - adAccountName: the name of the Active Directory 
#     account whose SPNs we want to obtain. 
# 
# Returns: the list of existing SPNs of the given
#           Active Directory account.
def getExistingSPNsOfAdAccountWindows(adAccountName):
  existingSPNsOfAdAccount = []

  powerShellListSPNsCommandStr = "setspn -L {0}".format(adAccountName)
  stdOut, stdErr = runPowerShellCommand(powerShellListSPNsCommandStr)

  # The first line of the output of the command above
  # will be a header line. The following is an example
  # of what such an output could look like:
  # 
  # Registered ServicePrincipalNames for CN=dwdms-compute-0-0,OU=aris,DC=domain,DC=local
  #       DWDMSSvc/compute-0-0.domain.local:16460
  #       DWDMSSvc/compute-0-0.domain.local
  if "Could not find account '{0}'".format(adAccountName) in stdErr:
    print("Error. Call to get SPNs failed because the account '{0}' was not found.".format(adAccountName))
  else:
    existingSPNsOfAdAccount = stdOut.split("\n")

    # Delete the first line of the output.
    # This is just a header that does not
    # actually include any SPNs.
    existingSPNsOfAdAccount.pop(0)

    # Delete extra whitespace in the strings.
    existingSPNsOfAdAccount = [spn.strip() for spn in existingSPNsOfAdAccount]

  return existingSPNsOfAdAccount


# Function: getExistingUsersInOULinux
# 
# Description: Calls ADUtil to get the
# list of existing users
# 
# Parameters:
#   ouDistinguishedName - the distinguished name of
#   the organizational unit used in the cluster deployment
# 
# Returns: the list of existing users in the organizational
# unit used in the cluster deployment.
def getExistingUsersInOULinux(ouDistinguishedName):
  global LOCAL_ERROR_IN_LDAP_CALL
  existingUsersList = []

  adutilGetUsersInOUCommandStr = "adutil ou getmembers --distname '{0}' --filter user".format(ouDistinguishedName)
  stdOut, stdErr = runBashCommand(adutilGetUsersInOUCommandStr)

  # We should look for this error message both in 'stdOut'
  # and 'stdErr' as adutil might be using either at the moment.
  # (5/29/2020) TODO: make adutil return error messages only to std err
  ldapCallErrorMessagePrefix = 'Failed to perform LDAP search'

  # The output of the execution of the command above will be a set
  # of lines of the following type:
  # 
  #       "User       CN=krbtgt,CN=Users,DC=stress,DC=local"
  # 
  # The users that we will want to parse are going to be the values
  # of the first occurrences of "CN=" in those lines. In the example,
  # that is "krbtgt". To parse that, we will find the first occurrence
  # of an equal sign ('=') from left to right and then the first occurrence
  # of a comma (',') from left to right as well. The user will then be
  # the substring after the equal sign ('=') and before the comma (',').
  if ldapCallErrorMessagePrefix in stdErr or ldapCallErrorMessagePrefix in stdOut:
    errorMessage = "Error. Call to get users of the OU '{0}' failed. ".format(ouDistinguishedName)

    noSuchObjectErrorMessageFragment = 'No such object'
    localErrorMessageFragment = 'Local error'

    if noSuchObjectErrorMessageFragment in stdErr or noSuchObjectErrorMessageFragment in stdOut:
      errorMessage += "No such OU exists."
    elif localErrorMessageFragment in stdErr or localErrorMessageFragment in stdOut:
      errorMessage += "It's likely that the credentials cache was destroyed before the LDAP calls were made."
      errorMessage += "Try performing the 'kinit' again."
      LOCAL_ERROR_IN_LDAP_CALL = True
    else:
      errorMessage += "Check the domain component (DC) part of the distinguished name of the OU."

    print(errorMessage)
  else:
    distinguishedNamesList = stdOut.split("\n")
    
    for distinguishedName in distinguishedNamesList:
      startingIndex = distinguishedName.find('=')
      startingIndex += 1
      endingIndex = distinguishedName.find(',')

      if startingIndex < endingIndex:
        user = distinguishedName[startingIndex:endingIndex]
        existingUsersList.append(user)
      else:
        print("Could not find the user in the following distinguished name: {0}.".format(distinguishedName))

  return existingUsersList


# Function: getExistingUsersInOUWindows
# 
# Description: calls PowerShell cmdlet to get the 
# list of users inside the organizational unit
# 
# Parameters:
#   ouDistinguishedName - the distinguished name of
#   the organizational unit used in the cluster deployment
# 
# Returns: the list of existing users in the organizational
# unit used in the cluster deployment.
def getExistingUsersInOUWindows(ouDistinguishedName):
  existingUsersList = []

  getADUsersInOrganizationalUnitPowerShellCommandStr = "Get-ADUser -Filter * -SearchBase '{0}' | Select -ExpandProperty Name".format(ouDistinguishedName)
  stdOut, stdErr = runPowerShellCommand(getADUsersInOrganizationalUnitPowerShellCommandStr)

  if stdErr != '':
    errorMessage = "Error. Call to get the users of the OU '{0}' failed. ".format(ouDistinguishedName)

    if 'Directory object not found' in stdErr:
      errorMessage += "No such OU exists."
    else:
      errorMessage += "Check the domain component (DC) part of the distinguished name of the OU."
    
    print(errorMessage)
  else:
    existingUsersList = stdOut.split()

  return existingUsersList


# Function: getExistingUsersInOU
# 
# Parameters:
#   ouDistinguishedName - the distinguished name of
#   the organizational unit used in the cluster deployment.
# 
# Returns: the list of existing users in the organizational
# unit used in the cluster deployment.
def getExistingUsersInOU(ouDistinguishedName):
  usersList = []

  if HOST_OS_NAME == LINUX_OS_NAME:
    usersList = getExistingUsersInOULinux(ouDistinguishedName)
  elif HOST_OS_NAME == WINDOWS_OS_NAME:
    usersList = getExistingUsersInOUWindows(ouDistinguishedName)

  return usersList


# Function: getExistingSPNsLinux
# 
# Parameters:
# 
# Returns: the list of existing SPNs.
def getExistingSPNs():
  existingSPNsList = []
  existingUsersList = getExistingUsersInOU(OU_DISTINGUISHED_NAME)

  for user in existingUsersList:
    userSPNs = []

    if HOST_OS_NAME == LINUX_OS_NAME:
      userSPNs = getExistingSPNsOfAdAccountLinux(user)
    elif HOST_OS_NAME == WINDOWS_OS_NAME:
      userSPNs = getExistingSPNsOfAdAccountWindows(user)

    existingSPNsList += userSPNs
  return existingSPNsList

:::

In [ ]:
# Perform SPNs checks.


# Function: spnsExist
# 
# Parameters:
# 
# Returns: True if the required SPNs exist. Else, false.
def spnsExist():
  # To check whether the SPNs exist, we'll do the following:
  # 
  # 1. Get the list of all existing users.
  # 2. Dump all existing SPNS of each existing user into a list.
  # 3. Generate the expected SPNs based on the bdc.json file and not
  #    on the expected users.
  # 4. Check whether the list of expected SPNs is a subset of the list
  #    of existing SPNs.
  # 
  # Note: this approach does not take into account which SPN
  # corresponds to which user. However, we consider that this
  # is fine at least for now.
  expectedSPNsList = getExpectedSPNs(DEPLOYMENT_COMPONENTS_LIST)
  existingSPNsList = getExistingSPNs()

  res = True

  for spn in expectedSPNsList:
    if spn not in existingSPNsList:
      res = False
      print("Error. SPN '{0}' was expected, but is not in the list of existing SPNs.".format(spn))

  return res

shouldRunTests = False
spnsChecksSucceeded = False

try:
  NOTEBOOK_EXECUTION_SETUP_SUCCEEDED
  shouldRunTests = True
except NameError:
  print("Error: The first cells of this notebook, which perform the notebook execution setup, must be run before this cell.")

if shouldRunTests:
  spnsChecksSucceeded = dependenciesAreInstalled() and spnsExist()

if spnsChecksSucceeded:
  print("SPNs checks were successful.")
else:
  print("Error. SPNs checks failed.")

:::

Summary
-------

In [ ]:
# Print summary of tests.


# Function: rightJustifyString
# 
# Parameters:
#   s - the string to right-justify
#   paddingSize - the amount of characters required for the padding
#   of the status string.
# 
# Returns: the right-justified string.
def rightJustifyString(s, paddingSize):
  totalStringLength = 4 + len(s) + paddingSize
  return s.rjust(totalStringLength)


# Define constants.
STATUS_SUCCEEDED = '\x1b[32mSUCCEEDED\x1b[0m'
STATUS_FAILED = '\x1b[31mFAILED\x1b[0m'
STATUS_NOT_RUN = 'NOT RUN'

# Define test names.
parseConfigurationFilesTestName = 'Parse Configuration Files'
dnsChecksTestName = 'DNS Checks'
adObjectsChecksTestName = 'AD Object Checks'
spnChecksTestName = 'SPNs Checks'

# Dictionary mapping test name to test status.
testNameToStatusDict = {}

# Test 1: Parse configuration files.
try:
  if loadedBdcJson and loadedControlJson:
    testNameToStatusDict[parseConfigurationFilesTestName] = STATUS_SUCCEEDED
  else:
    testNameToStatusDict[parseConfigurationFilesTestName] = STATUS_FAILED
except NameError:
  testNameToStatusDict[parseConfigurationFilesTestName] = STATUS_NOT_RUN

# Test 2: DNS checks.
try:
  if dnsChecksSucceeded:
    testNameToStatusDict[dnsChecksTestName] = STATUS_SUCCEEDED
  else:
    testNameToStatusDict[dnsChecksTestName] = STATUS_FAILED
except NameError:
  testNameToStatusDict[dnsChecksTestName] = STATUS_NOT_RUN


# Test 3: AD object checks.
try:
  if adAccountsChecksSucceeded:
    testNameToStatusDict[adObjectsChecksTestName] = STATUS_SUCCEEDED
  else:
    testNameToStatusDict[adObjectsChecksTestName] = STATUS_FAILED
except NameError:
  testNameToStatusDict[adObjectsChecksTestName] = STATUS_NOT_RUN

# Test 4: SPN checks.
try:
  if spnsChecksSucceeded:
    testNameToStatusDict[spnChecksTestName] = STATUS_SUCCEEDED
  else:
    testNameToStatusDict[spnChecksTestName] = STATUS_FAILED
except NameError:
  testNameToStatusDict[spnChecksTestName] = STATUS_NOT_RUN

# Get the length of the longest test name.
maxTestNameLength = 0
if len(testNameToStatusDict) > 0:
  maxTestNameLength = max([len(testName) for testName in testNameToStatusDict.keys()])

allTestsSucceeded = True

# Iterate on the dictionary of tests.
for testName in testNameToStatusDict.keys():
  testStatus = testNameToStatusDict[testName]
  allTestsSucceeded = allTestsSucceeded and (STATUS_SUCCEEDED == testStatus)
  testNamePadding = maxTestNameLength - len(testName)

  print("{0}:{1}".format(testName, rightJustifyString(testStatus, testNamePadding)))

# Throw an exception if not all tests succeeded and no LDAP call failed.
if allTestsSucceeded:
  print("Success. All tests passed.")
else:
  if LOCAL_ERROR_IN_LDAP_CALL:
    print("It's likely that a separate GCI test ran 'kdestroy' before the last LDAP call made by this notebook through adutil.")
  else:
    raise Exception("Not all tests passed.")

:::

In [ ]:
print("Notebook execution is complete.")